In [1]:
# install packages as needed
! pip install sentence-transformers
! pip install numpy
! pip install pandas
! pip install sklearn

In [2]:
# libraries
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/tuo70125/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-30 15:42:13.130083: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# load pre-trained sentence transformer model (aka SBERT)
model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/tuo70125/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# load pilot annotation data
data = pd.read_csv("/Users/tuo70125/My Drive/SANLab/Experiments/CANDOR/analysis/data/data-for-python.csv")
# print preview of data
data.head()

,scaled_turn_id,new_topic,utterance,speaker,transcript_id
0,0.0,NaN,Yeah. Yeah. One. All right. Right. Uh huh. Mhm...,5b18647968a5ed00017343ec,b8ca03fb-ba34-49dc-b33f-30b6e9189685
1,0.0,NaN,mm hmm. Yeah. Yeah. Mm hmm. Mhm. Mhm. Okay. Wh...,5e121b2b82a955890a9b3484,b2afb6c2-6049-4861-88a5-f0aeba68b5c8
2,0.0,NaN,"Yeah. Mhm Okay. Mhm, wow. Same thing, yeah. Ye...",5e13ad514c81b2a01ae4aa9f,acd5c3b5-c080-4ab4-98ca-4099c2711591
3,0.0,NaN,"Hey, I made it first. That's good. Will anyone...",5eaa26c17c3cbc1ae4881baa,ec397cba-60f5-4bc5-ab02-73a2aaa17029
4,0.0,NaN,Uh huh. Mhm Yeah. Okay. He did. Yeah. Yeah. Yo...,5eb4a3fb75168630c3e6fdb1,f7ddd168-2607-4d2b-b01d-905e4e814ba6


## Cosine similarity between utterance and topic

In [5]:
# make sure chat is pandas dataframe
chat = pd.DataFrame(data)

# create function to calculate cosine similarity between annotated topic and utterance
def question_chat_cosine_similarity(row):
    # extract current message and current topic
    current = [row['utterance'], row['new_topic']]
    # apply s-bert model to each string to get embeddings
    embeddings = model.encode(current)
    # calculate cosine similarity between question and message
    similarity = model.similarity(embeddings, embeddings)
    return similarity[0][1]

# apply function to each row of chat data
chat['topic_utterance_similarity'] = chat.apply(question_chat_cosine_similarity, axis = 1)

In [6]:
# save
chat.to_csv('/Users/tuo70125/My Drive/SANLab/Experiments/CANDOR/analysis/data/utterance-topic-cosine.csv')
# print 
chat.head()

,scaled_turn_id,new_topic,utterance,speaker,transcript_id,topic_utterance_similarity
0,0.0,NaN,Yeah. Yeah. One. All right. Right. Uh huh. Mhm...,5b18647968a5ed00017343ec,b8ca03fb-ba34-49dc-b33f-30b6e9189685,tensor(0.1831)
1,0.0,NaN,mm hmm. Yeah. Yeah. Mm hmm. Mhm. Mhm. Okay. Wh...,5e121b2b82a955890a9b3484,b2afb6c2-6049-4861-88a5-f0aeba68b5c8,tensor(0.1982)
2,0.0,NaN,"Yeah. Mhm Okay. Mhm, wow. Same thing, yeah. Ye...",5e13ad514c81b2a01ae4aa9f,acd5c3b5-c080-4ab4-98ca-4099c2711591,tensor(0.1799)
3,0.0,NaN,"Hey, I made it first. That's good. Will anyone...",5eaa26c17c3cbc1ae4881baa,ec397cba-60f5-4bc5-ab02-73a2aaa17029,tensor(0.0702)
4,0.0,NaN,Uh huh. Mhm Yeah. Okay. He did. Yeah. Yeah. Yo...,5eb4a3fb75168630c3e6fdb1,f7ddd168-2607-4d2b-b01d-905e4e814ba6,tensor(0.1107)
